In [10]:
import matplotlib.pyplot as plt
import numpy as np
from qulacs import QuantumState
import sklearn
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

def show_figure(pict: np.array):
    plt.imshow(pict.reshape(28, 28), cmap="Greys")
    plt.show()


def show_state_figure(state: QuantumState):
    pict = state.get_vector()
    result = []
    for x in range(28 * 28):
        result.append(pict[x].real)
    show_figure(np.array(result))

def show_vector_figure(pict):
    result = []
    for x in range(28 * 28):
        result.append(pict[x].real)
    show_figure(np.array(result))

In [13]:
from mnisq.mnist import load_large_f80 as mnist_large_f80 
from mnisq.fashion_mnist import load_large_f80 as fashion_mnist_large_f80 
from mnisq.kuzushiji_mnist import load_large_f80 as kuzushiji_mnist_large_f80 
from mnisq.mnist import load_small_f80 as mnist_small_f80 
from mnisq.fashion_mnist import load_small_f80 as fashion_mnist_small_f80 
from mnisq.kuzushiji_mnist import load_small_f80 as kuzushiji_mnist_small_f80 
from mnisq.mnist import load_large_f90 as mnist_large_f90 
from qulacs.state import inner_product
import numpy as np
from tqdm import tqdm

# fのあとに続く数字が大きいほど、精度と量子回路のサイズが大きい。
# 80,90,95というチョイスは今のところ意味はない。

dataset_list = [
  #{
  #  "name": "kuzushiji_mnist_large_f80",
  #  "loader": kuzushiji_mnist_large_f80
  #},
  #{
  #  "name": "fashion_mnist_large_f80",
  #  "loader": fashion_mnist_large_f80
  #},
  {
    "name": "mnist_large_f90",
    "loader": mnist_large_f90
  }
]

for dataset in dataset_list:
  data = dataset["loader"]()
  labels = data["label"][:20000] # ラベル (integerの配列)
  circuits = data["circuit"][:20000] # 回路 (QuantumCircuitの配列)
  states = data["state"][:20000] # 回路にAQCEで変換する前の量子状態
  qasms = data["qasm"][:20000] # 回路のQASM文字列
  
  del data
  
  labels = np.array(labels, dtype=int)
  indices = np.where((labels!=-1))[0]
  x_all = np.zeros((len(indices), len(states[0].get_vector())))
  y_all = np.zeros(len(indices), dtype=int)
  quantum_x_all = np.zeros((len(indices), len(states[0].get_vector())))
  state = states[0].copy()
  for i,j in enumerate(indices):
      x_all[i] = states[j].get_vector()
      state.set_zero_state()
      circuits[j].update_quantum_state(state)
      quantum_x_all[i] = state.get_vector()
      y_all[i] = labels[j]

  #x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.4, random_state=0)
  q_x_train, q_x_test, q_y_train, q_y_test = train_test_split(quantum_x_all, y_all, test_size=0.4, random_state=0)
  
  # use the same circuit as scikit-qulacs do.
  clf = SVC(kernel="precomputed")
  print("calculating kernel")
  train_kernel = np.zeros((len(q_x_train), len(q_x_train)))
  for i in tqdm(range(len(q_x_train))):
    for j in range(i,len(q_x_train)):
      train_kernel[i][j] = abs(np.vdot(q_x_train[i],q_x_train[j])) ** 2
      train_kernel[j][i] = train_kernel[i][j]
  
  clf.fit(train_kernel, q_y_train)
  
  test_kernel = np.zeros((len(q_x_test), len(q_x_train)))
  for i in tqdm(range(len(q_x_test))):
    for j in range(len(q_x_train)):
      test_kernel[i][j] = abs(np.vdot(q_x_test[i],q_x_train[j])) ** 2
  pred = clf.predict(test_kernel)
  print(f"{dataset['name']} Test Accuracy : ", (pred == q_y_test).mean())
  
  del labels, circuits, states, qasms

100%|██████████| 60000/60000 [00:01<00:00, 36385.91it/s]
/tmp/ipykernel_1035/318562461.py:46: ComplexWarning: Casting complex values to real discards the imaginary part
  x_all[i] = states[j].get_vector()
/tmp/ipykernel_1035/318562461.py:49: ComplexWarning: Casting complex values to real discards the imaginary part
  quantum_x_all[i] = state.get_vector()


calculating kernel


100%|██████████| 8000/8000 [02:26<00:00, 54.43it/s]


mnist_large_f90 Test Accuracy :  0.96125


In [12]:
from mnisq.mnist import load_large_f80 as mnist_large_f80 
from mnisq.fashion_mnist import load_large_f80 as fashion_mnist_large_f80 
from mnisq.kuzushiji_mnist import load_large_f80 as kuzushiji_mnist_large_f80 
from mnisq.mnist import load_small_f80 as mnist_small_f80 
from mnisq.fashion_mnist import load_small_f80 as fashion_mnist_small_f80 
from mnisq.kuzushiji_mnist import load_small_f80 as kuzushiji_mnist_small_f80 
from qulacs.state import inner_product
import numpy as np
from tqdm import tqdm

for dataset in dataset_list:
  data = dataset["loader"]()
  labels = data["label"][:20000] # ラベル (integerの配列)
  circuits = data["circuit"][:20000] # 回路 (QuantumCircuitの配列)
  states = data["state"][:20000] # 回路にAQCEで変換する前の量子状態
  qasms = data["qasm"][:20000] # 回路のQASM文字列
  
  del data
  
  labels = np.array(labels, dtype=int)
  indices = np.where((labels!=-1))[0]
  x_all = np.zeros((len(indices), len(states[0].get_vector())))
  y_all = np.zeros(len(indices), dtype=int)
  quantum_x_all = np.zeros((len(indices), len(states[0].get_vector())))
  state = states[0].copy()
  for i,j in enumerate(indices):
      x_all[i] = states[j].get_vector()
      #state.set_zero_state()
      #circuits[j].update_quantum_state(state)
      #quantum_x_all[i] = state.get_vector()
      y_all[i] = labels[j]

  x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.4, random_state=0)
  q_x_train, q_x_test, q_y_train, q_y_test = train_test_split(quantum_x_all, y_all, test_size=0.4, random_state=0)
  
  # use the same circuit as scikit-qulacs do.
  clf = SVC(kernel="linear")
  
  clf.fit(x_train, y_train)
  
  pred = clf.predict(x_test)
  print(f"{dataset['name']} Test Accuracy : ", (pred == y_test).mean())
  
  del labels, circuits, states, qasms

KeyboardInterrupt: 